## Model training and deployment to Azure

#### Installation of accessory libraries

In [1]:
import pycaret 
print(pycaret.__version__)

3.0.4


In [2]:
!pip install azure-storage-blob

  Using cached azure_storage_blob-12.17.0-py3-none-any.whl (388 kB)
  Using cached azure_core-1.29.3-py3-none-any.whl (191 kB)
  Using cached isodate-0.6.1-py2.py3-none-any.whl (41 kB)


In [4]:
!pip install git+https://github.com/amjadraza/pycaret.git@feature/gcp_azure_np_docs

  Cloning https://github.com/amjadraza/pycaret.git (to revision feature/gcp_azure_np_docs) to c:\users\caplogy_data_002\appdata\local\temp\pip-req-build-0vtmc6o5
  Resolved https://github.com/amjadraza/pycaret.git to commit b583d082a8be33f9d7afde634fd1b27847c33f6c
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached yellowbrick-1.5-py3-none-any.whl (282 kB)
  Using cached xgboost-1.7.6-py3-none-win_amd64.whl (70.9 MB)
  Using cached lightgbm-4.0.0-py3-none-win_amd64.whl (1.3 MB)
  Using cached wordcloud-1.9.2-cp311-cp311-win_amd64.whl (151 kB)
  Using cached textblob-0.17.1-py2.py3-none-any.whl (636 kB)
  Using cached cufflinks-0.17.3.tar.gz (81 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached umap-learn-0.5.3.tar.gz (88 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pyLDAvis-3.4

  Running command git clone --filter=blob:none --quiet https://github.com/amjadraza/pycaret.git 'C:\Users\Caplogy_Data_002\AppData\Local\Temp\pip-req-build-0vtmc6o5'
  Running command git checkout -b feature/gcp_azure_np_docs --track origin/feature/gcp_azure_np_docs
  branch 'feature/gcp_azure_np_docs' set up to track 'origin/feature/gcp_azure_np_docs'.
  Switched to a new branch 'feature/gcp_azure_np_docs'
  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [18 lines of output]
  The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
  rather than 'sklearn' for pip commands.
  
  Here is how to fix this error in the main use cases:
  - use 'pip install scikit-learn' rather than 'pip install sklearn'
  - replace 'sklearn' by 'scikit-learn' in your pip requirements files
    (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
  - if the 'sklearn' package is used by one of your dependencies,
    it would be great i

### Model Development

In [8]:
from pycaret.datasets import get_data
dataset = get_data('diamond')

,Carat Weight,Cut,Color,Clarity,Polish,Symmetry,Report,Price
0,1.10,Ideal,H,SI1,VG,EX,GIA,5169
1,0.83,Ideal,H,VS1,ID,ID,AGSL,3470
2,0.85,Ideal,H,SI1,EX,EX,GIA,3183
3,0.91,Ideal,E,SI1,VG,VG,GIA,4370
4,0.83,Ideal,G,SI1,EX,EX,GIA,3171


In [9]:
data = dataset.sample(frac=0.95, random_state=786).reset_index(drop=True)
data_unseen = dataset.drop(data.index).reset_index(drop=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

Data for Modeling: (5700, 8)
Unseen Data For Predictions: (300, 8)


In [11]:
from pycaret.regression import *
exp_reg101 = setup(
    data = data, 
    target = 'Price', 
    session_id=123,
    #silent = True
                  )

,Description,Value
0,Session id,123
1,Target,Price
2,Target type,Regression
3,Original data shape,"(5700, 8)"
4,Transformed data shape,"(5700, 29)"
5,Transformed train set shape,"(3989, 29)"
6,Transformed test set shape,"(1711, 29)"
7,Ordinal features,1
8,Numeric features,1
9,Categorical features,6


In [12]:
lightgbm = create_model('lightgbm')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,764.4776,3694398.5693,1922.0818,0.9700,0.0729,0.0524
1,951.5729,7114848.9941,2667.3674,0.9425,0.0843,0.0588
2,853.0754,5146743.2184,2268.6435,0.9578,0.0834,0.0610
3,699.4826,1730518.0345,1315.4916,0.9811,0.0766,0.0557
4,633.4070,1505126.5084,1226.8360,0.9852,0.0681,0.0523
5,758.1128,2698170.1978,1642.6108,0.9727,0.0784,0.0581
6,846.4091,6154298.7686,2480.7859,0.9401,0.0810,0.0583
7,729.4342,2328849.0908,1526.0567,0.9728,0.0765,0.0560
8,837.5604,2614949.9435,1617.0807,0.9738,0.0824,0.0622


In [13]:
tuned_lightgbm = tune_model(lightgbm)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,863.4824,4070841.3163,2017.6326,0.9669,0.1043,0.0720
1,1038.3901,6851142.9179,2617.4688,0.9446,0.1047,0.0754
2,1039.7331,7162511.3618,2676.2869,0.9413,0.1115,0.0811
3,797.3442,1979336.8041,1406.8891,0.9784,0.0983,0.0696
4,923.7252,3117836.1058,1765.7395,0.9694,0.1060,0.0777
5,925.6945,3046045.0913,1745.2923,0.9692,0.1292,0.0841
6,980.9006,7715048.3913,2777.5976,0.9249,0.1185,0.0736
7,858.1841,3017677.4242,1737.1463,0.9648,0.1145,0.0723
8,892.6893,2789277.3724,1670.1130,0.9721,0.0876,0.0675


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [14]:
predict_model(tuned_lightgbm );

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,712.3047,2288865.2634,1512.8996,0.9776,0.0746,0.0544


### Finalize the model

In [15]:
final_lightgbm  = finalize_model(tuned_lightgbm )

In [16]:
print(final_lightgbm)

Pipeline(memory=FastMemory(location=C:\Users\CAPLOG~1\AppData\Local\Temp\joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['Carat Weight'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=['Cut', 'Color', 'Clarity',
                                             'Polish', 'Symmetry', 'Report'],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('ordinal...
                ('onehot_encoding',
                 TransformerWrapper(include=['Cut', 'Color', 'Clarity',
                                             'Polish', 'Symmetry'],
                                    transformer=OneHotEncoder(cols=['Cut',
                                                                    'Color',
                                                                    'Clarity',
                                  

In [17]:
predict_model(final_lightgbm);

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,532.1047,880939.6565,938.5839,0.9914,0.0625,0.0463


# Deploy the model to Microsoft Azure

This is the code to implement the model in Microsoft Azure using the `pycaret` features.

In [18]:
def create_container(container_name):

  # Create the container
    container_client = blob_service_client.create_container(container_name)

    return container_client

def upload_blob(container_name, source_file_name, destination_blob_name):

  # Create a blob client using the local file name as the name for the blob
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=destination_blob_name)

    print("\nUploading to Azure Storage as blob:\n\t" + source_file_name)

  # Upload the created file
    with open(source_file_name, "rb") as data:
        blob_client.upload_blob(data)

def download_blob(container_name, source_blob_name, destination_file_name):
  # Download the blob to a local file
    print("\nDownloading blob to \n\t" + destination_file_name)

  # Create a blob client using the local file name as the name for the blob
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=source_blob_name)

    if destination_file_name is not None: 
        with open(destination_file_name, "wb") as download_file:
            download_file.write(blob_client.download_blob().readall())

        print(
            "Blob {} downloaded to {}.".format(
                source_blob_name, destination_file_name
            )
        )

In [19]:
import os, uuid
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

#### Here you should put the access token (Connection String)

In [20]:
## Enter connection string when running in google colab
connect_str = 'DefaultEndpointsProtocol=https;AccountName=machinelearninginsurance;AccountKey=t1ocBOnBT1jKNmfVa0j9TN///NfkxS0fjucCK3KG4SCcqHDBXIyceHU/SYjj0Odmrp1jX/Jvd0ZR+AStVL/6cA==;EndpointSuffix=core.windows.net' 
print(connect_str)

DefaultEndpointsProtocol=https;AccountName=machinelearninginsurance;AccountKey=t1ocBOnBT1jKNmfVa0j9TN///NfkxS0fjucCK3KG4SCcqHDBXIyceHU/SYjj0Odmrp1jX/Jvd0ZR+AStVL/6cA==;EndpointSuffix=core.windows.net


In [21]:
# Create the BlobServiceClient object which will be used to create a container client
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

##### Create the container in the Blob Storage

In [22]:
# Create a unique name for the container
container_name = "machinelearninginsurance"
container_client = create_container(container_name)
model_dir = 'modelo'

In [23]:
# Save Model Local and upload to Azure
model_name_azure = 'lightgbm-reg-101'
save_model(final_lightgbm, 
           model_name= model_name_azure, 
           verbose=False)

model_src = model_name_azure +'.pkl'
model_dst = str(model_name_azure)+'.pkl'

##### Upload the model

In [24]:
upload_blob(container_name, model_src, model_dst)


Uploading to Azure Storage as blob:
	lightgbm-reg-101.pkl
